In [1]:
import pandas as pd
import numpy as np
import os
import sys
import datetime

In [2]:
sys.path.append(os.path.join(os.getcwd(), '..', 'utils'))
from course_id_all_files import get_files_with_course_code, get_files_with_course_code2 # Hàm tìm kiếm tên file bằng mã học phần

In [3]:
current_directory = os.getcwd()
folder_path1 = os.path.join(current_directory,'../../data/gathered_data/attendance_sheet')
folder_path2 = os.path.join(current_directory,'../../data/gathered_data/score_table')
folder_path3 = os.path.join(current_directory,'../../data/reformatted_data')
file_names1 = os.listdir(folder_path1)
file_names2 = os.listdir(folder_path2)

In [4]:
""" Bảng subject """
# Số tín chỉ
credits = pd.read_excel('../crawl/Score_Data.xlsx')
credits_infor = (credits.loc[credits['Mã số sinh viên']==2000001001]).iloc[:,2:6]
# Đổi tên cột của bảng tín chỉ
credits_infor.columns = ['student id','course id','subject name', 'credit']
credits_infor.to_csv('../../data/reformatted_data/subject.csv', index=False)
credits_infor['course id'] = credits_infor['course id'].astype('object')

subject_names = credits_infor['subject name']
subjects_table = pd.DataFrame(subject_names)
subjects_table['subject id'] = subjects_table.index
subjects_table = subjects_table.reindex(columns=['subject id', 'subject name'])
subjects_table['subject id'] = subjects_table['subject id'].astype('object')
new_row = pd.DataFrame({'subject id': ['48'], 'subject name': ['Thiết kế và phát triển ứng dụng Web']})
subjects_table = pd.concat([subjects_table, new_row], ignore_index=True)
subjects_table = subjects_table.drop(subjects_table[subjects_table['subject id'] == 20].index)
subjects_table.to_csv('../../data/reformatted_data/subject.csv', index=False,mode='w')

In [5]:
subjects_table

,subject id,subject name
0,0,Tiếng Anh 2
1,1,Toán cao cấp (CNTT)
2,2,Dịch vụ Hành chính
3,3,Lập trình
4,4,Toán rời rạc
5,5,Công nghệ thông tin
6,6,Tiếng Anh 3
7,7,Giáo dục thể chất
8,8,Kỹ năng giao tiếp
9,9,Pháp luật đại cương


In [6]:
# Hàm lấy thông tin học phần trong file điểm danh
def get_cource_infor(path):
    df = pd.read_excel(path, nrows = 13)
    # Tổng số tiết học.
    total_class_periods = df.iloc[12, -2]

    time_ = df.iloc[10,6:df.shape[1]-4].dropna()
    start_time = time_.iloc[0][-10:]
    end_time = time_.iloc[-1][-10:]
    # batch, year
    batch_year = df.iloc[4,2]
    batch_year_elements = batch_year.split(' ')
    batch = batch_year_elements[1]
    year = batch_year_elements[3]
    date_series = df.iloc[10,6:df.shape[1]-4].dropna()
    # Số tiết học.
    school_periods = date_series.shape[0]
    # class name
    cource_infor = df.iloc[7, 2]
    class_name = cource_infor[-9:len(cource_infor)-1]
    # cource code
    cource_code = cource_infor[-24:-12]
    # cource  name
    cource_name = cource_infor[0:len(cource_infor)-(len(cource_code)+len(class_name) + 6)]

    return (cource_name, class_name, cource_code,school_periods, batch, year, start_time, end_time,total_class_periods)


# Hàm lấy thông tin học phần trong file điểm sinh viên
def get_cource_infor_s(path):
    df = pd.read_excel(path, nrows = 8)
    # batch, year
    batch_year = df.iloc[3,1]
    batch_year_elements = batch_year.split(' ')
    batch = batch_year_elements[2]
    year = batch_year_elements[4]
    course_infor = (df.iloc[3,4])[14:]
    parts = course_infor.split(" - ")
    # cource id
    course_id = parts[0].strip('[]')
    # class name
    class_id = parts[1][-9:len(parts[1])-1]
    # subject name
    subject = parts[1][:len(parts[1])-11]
    return (course_id, class_id, subject, batch, year)

In [7]:
""" Bảng class """
# ---------------------------------- TẠO DANH SÁCH LỚP HỌC TỪ FILE ĐIỂM DANH-------------------------------------------------COMPLETE
class_names_ls = []
for i in file_names1:
    class_names_ls.append(get_cource_infor(f'{folder_path1}/{i}')[1])
# Sắp xếp lại index của series class_names bằng cách chuyển nó thành dataframe bằng pt reset_index(), sau đó sắp xếp lại theo quy tắc (2 chữ cái đầu, 1 chữ cái cuối)
# (np.array([[i for i in range(class_names_array.size)], list(class_names_array)]).T): Tạo mảng 2 chiều sau đó chuyển vị cột thành dòng, dòng thành cột
class_names = pd.Series(class_names_ls).value_counts()
class_names = class_names.reset_index().sort_values(by = ['index'], key=lambda x: x.str[:2] + x.str[-1])
class_names_array = class_names['index'].values # Lấy ra giá trị của cột index.
class_table = pd.DataFrame((np.array([[i for i in range(class_names_array.size)], list(class_names_array)]).T), columns= ['class_id', 'class_name'])
class_table1 = class_table.drop_duplicates()

# ---------------------------------- TẠO DANH SÁCH LỚP HỌC TỪ FILE ĐIỂM SINH VIEN-------------------------------------------------COMPLETE
class_names_ls = []
for i in file_names2:
    class_names_ls.append(get_cource_infor_s(f'{folder_path2}/{i}')[1])
# Sắp xếp lại index của series class_names bằng cách chuyển nó thành dataframe bằng pt reset_index(), sau đó sắp xếp lại theo quy tắc (2 chữ cái đầu, 1 chữ cái cuối)
# (np.array([[i for i in range(class_names_array.size)], list(class_names_array)]).T): Tạo mảng 2 chiều sau đó chuyển vị cột thành dòng, dòng thành cột
class_names = pd.Series(class_names_ls).value_counts()
class_names = class_names.reset_index().sort_values(by = ['index'], key=lambda x: x.str[:2] + x.str[-1])
class_names_array = class_names['index'].values # Lấy ra giá trị của cột index.
class_table = pd.DataFrame((np.array([[i for i in range(class_names_array.size)], list(class_names_array)]).T), columns= ['class_id', 'class_name'])
class_table2 = class_table.drop_duplicates()

# ---------------------------------- TẠO DANH SÁCH LỚP HỌC------------------------------------------------COMPLETE

class_table = pd.concat([class_table1,class_table2])
class_table = class_table.drop_duplicates(ignore_index=True)
class_table['class_id'] = class_table.index
class_table.columns = ['class id', 'class name']
class_table['class id'] = class_table['class id'].astype('object')
class_table

,class id,class name
0,0,20BITV01
1,1,20BITV02
2,2,21BITV01
3,3,21BITV02
4,4,21BITV03
5,5,22BITV02
6,6,22BITV03
7,7,22BITV04
8,8,22BITV03
9,9,22BITV04


In [8]:
""" Bảng student 1"""
# ---------------------------------- TẠO DANH SÁCH SINH VIÊN (BẢNG ĐIỂM DANH)----------------------------------------------COMPLETE

list_df = []
for i in file_names1:
    students_table = pd.read_excel(f'../../data/gathered_data/attendance_sheet/{i}', skiprows=12, usecols='B:F')
    students_table['Unnamed: 6'] = students_table['Unnamed: 1'].apply(lambda x: str(x)[:2])
    list_df.append(students_table)
students_table = pd.concat(list_df, ignore_index=True)

# ---------------------------------- TẠO DANH SÁCH SINH VIÊN (BẢNG ĐIỂM) -------------------------------------------------COMPLETE
student_data = []
for i in file_names2:                         
    df = (pd.read_excel(f'{folder_path2}/{i}', skiprows=11, usecols='B:F')).dropna()
    df['Unnamed: 6'] = df['Unnamed: 1'].apply(lambda x: str(x)[:2])
    student_data.append(df)
students_table2 = pd.concat(student_data,ignore_index=True)


# ---------------------------------- GỘP 2 BẢNG --------------------------------------------------------------------------COMPLETE\
students_table3 = pd.concat([students_table,students_table2])
students_table3.columns = ['student_id', 'first name', 'last name', 'gender', 'dob', 'school year']
students_table3 = students_table3.replace({'Nam': 1, 'Nữ':0})
arr1_str = students_table['Unnamed: 1'].astype(str)
arr2_str = students_table2['Unnamed: 1'].astype(str)
result = pd.DataFrame(np.intersect1d(arr1_str, arr2_str))
result_df = pd.DataFrame(result)
result_df.columns = ['student_id']
final_students = students_table3[students_table3['student_id'].isin(result_df['student_id'])].drop_duplicates(keep='first')
final_students = final_students.reset_index(drop=True)
final_students.to_csv('../../data/reformatted_data/student.csv',index=False,mode='w')
final_students


C:\Users\HP\AppData\Local\Temp\ipykernel_50244\3620311551.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  students_table3 = students_table3.replace({'Nam': 1, 'Nữ':0})


,student_id,first name,last name,gender,dob,school year
0,2000001094,Đoàn Tuấn,Anh,1,16/12/2002,20
1,2000001008,Thái Mai Hoàng,Anh,1,15/10/2002,20
2,2000004387,Trần Thanh,Bình,1,07/12/2002,20
3,2000002380,Huỳnh Hoàng,Duy,1,16/12/2002,20
4,2000003127,Phạm Trần Quốc,Duy,1,01/09/2002,20
...,...,...,...,...,...,...
180,2200011573,Tạ Quang,Thành,1,13/10/2004,22
181,2200009571,NGUYỄN HUỲNH PHÚC,THỌ,1,09/09/2003,22
182,2200009501,Nguyễn Văn,Thuận,1,06/05/2004,22
183,2200010286,Phan Minh,Thuận,1,06/09/2004,22


In [9]:
""" Bảng student 2"""
# ---------------------------------- TẠO DANH SÁCH SINH VIÊN (BẢNG ĐIỂM DANH)----------------------------------------------COMPLETE

list_df = []
for i in file_names1:
    students_table = pd.read_excel(f'{folder_path1}/{i}', skiprows=12, usecols='B:F')
    students_table['Unnamed: 6'] = students_table['Unnamed: 1'].apply(lambda x: str(x)[:2])
    list_df.append(students_table)
students_table = pd.concat(list_df, ignore_index=True)
students_table['Unnamed: 1'] = students_table['Unnamed: 1'].astype('str')
students_table['Unnamed: 1'] = students_table['Unnamed: 1'].str.strip(' ')
students_table['Unnamed: 5'] = pd.to_datetime(students_table['Unnamed: 5']).dt.strftime('%d/%m/%Y')
students_table['Unnamed: 5'] = students_table['Unnamed: 5'].astype('object')

# ---------------------------------- TẠO DANH SÁCH SINH VIÊN (BẢNG ĐIỂM) -------------------------------------------------COMPLETE
student_data = []
for i in file_names2:                         
   students_table2 = (pd.read_excel(f'{folder_path2}/{i}', skiprows=10, usecols='B:F')).dropna()
   students_table2.columns = [f'Unnamed: {i+1}' for i in range(students_table2.shape[1])]
   students_table2['Unnamed: 1'] = students_table2['Unnamed: 1'].str.strip(' ')
   students_table2['Unnamed: 6'] = students_table2['Unnamed: 1'].apply(lambda x: str(x)[:2])
   student_data.append(students_table2)
students_table2 = pd.concat(student_data,ignore_index=True)


# # ---------------------------------- GỘP 2 BẢNG --------------------------------------------------------------------------COMPLETE\
final_students = pd.concat([students_table,students_table2])
final_students.columns = ['student_id', 'first name', 'last name', 'gender', 'dob', 'school year']
final_students['gender'] = final_students['gender'].replace({'Nam': 1, 'Nữ': 0})
final_students = final_students.drop_duplicates(keep = 'first', subset=['student_id'])
# (final_students['student_id'].value_counts() > 1).all()

C:\Users\HP\AppData\Local\Temp\ipykernel_50244\3342012822.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_students['gender'] = final_students['gender'].replace({'Nam': 1, 'Nữ': 0})


In [10]:
final_students

,student_id,first name,last name,gender,dob,school year
0,2000001008,Thái Mai Hoàng,Anh,1,15/10/2002,20
1,2000004387,Trần Thanh,Bình,1,07/12/2002,20
2,2000000998,Võ Thành,Công,1,02/07/2001,20
3,2000002380,Huỳnh Hoàng,Duy,1,16/12/2002,20
4,2000003127,Phạm Trần Quốc,Duy,1,01/09/2002,20
...,...,...,...,...,...,...
271,2200002076,Võ Duy,Toàn,1,27/07/2004,22
272,2200010216,LÊ MINH,TRÍ,1,12/08/2004,22
273,2200009170,LÊ THANH,TRỌNG,1,10/06/2003,22
274,2200010939,Lê Đoàn Anh,Tuấn,1,25/09/2004,22


In [11]:
""" Bảng course """
def get_course_infor(path):
    df = pd.read_excel(path, usecols='A:G')
    df = df.head(df.shape[0]-3)
    parts =  df.iloc[3,4].split(' - ')
    part1 = parts[0]
    part2 = parts[1]
    part3 = part2.split(' (')
    course_id = part1.split(': ')[1].strip('[]')
    subject_name = part3[0]
    class_name = part3[1].strip(')')
    return [course_id, subject_name, class_name]
# ----------------------------------DANH SÁCH HỌC PHẦN (BẢNG ĐIỂM DANH)------------------------------------------
course_data = [] # List of list
for i in file_names1:
    course_infor = get_cource_infor(f'../../data/gathered_data/attendance_sheet/{i}')
    course_code = course_infor[2]
    subject_title = course_infor[0]
    class_name = course_infor[1] 
    school_periods = course_infor[3]
    start_time = course_infor[6]
    end_time = course_infor[7]
    total_class_periods = course_infor[8]
    for j in range(len(file_names1)): # Thêm danh sách con vào trong danh sách cha
        child_course_data = [course_code,subject_title,class_name,school_periods, start_time, end_time]
        course_data.append(child_course_data)

course_df = pd.DataFrame(course_data, columns=['course id','subject name','class name','noc', 'start time', 'end time']) # noc: number of classes
course_df = course_df.drop_duplicates().sort_values(by=['course id']).reset_index(drop=True)

# ----------------------------------DANH SÁCH HỌC PHẦN (BẢNG ĐIỂM)---------------------------------------------------
course_data = [] # List of list
for i in file_names2:
    course_detail_data = (get_course_infor(f'{folder_path2}/{i}'))
    course_data.append(course_detail_data)
course_df1 = pd.DataFrame(course_data, columns=['course id','subject name','class name'])

# Tìm những course id chưa có trong course df
course_array = np.setdiff1d(course_df1['course id'].values, course_df['course id'].values)

course_df1 = course_df1[course_df1['course id'].isin(course_array)]
course_df1['noc'] = [np.NaN for i in range(course_df1.shape[0])]
course_df1['start time'] = [np.NaN for i in range(course_df1.shape[0])]
course_df1['end time'] = [np.NaN for i in range(course_df1.shape[0])]


# ----------------------------------GỘP 2 DF (BẢNG ĐIỂM VÀ BẢNG ĐIỂM DANH 'DF1,DF2')---------------------------------------------------

course_df3 = pd.concat([course_df, course_df1], ignore_index=True)

# ----------------------------------TÌM RA SỐ TÍN CHỈ CÓ TRONG CÁC MÔN HỌC CỦA COURSE DF ------------------------------------------------
course_code_array = np.array(course_df.iloc[:,1])
credits_infor_array = np.array(credits_infor.iloc[:,2])
credits_infor_array = credits_infor_array.astype(str)
# Tìm các môn học có trong cả hai mảng  
common_subjects = np.intersect1d(course_code_array, credits_infor_array)
# Lấy ra các môn học trùng khớp từ DataFrame credits_infor
matched_subjects_info = credits_infor[credits_infor['subject name'].isin(common_subjects)].reset_index(drop = True)
# Merge course_df, matched_subjects_info
final_course_df = pd.merge(course_df3, matched_subjects_info.iloc[:, [2,3]], on='subject name', how='left')
final_course_df['credit'] = final_course_df['credit'].astype('float16')
final_course_df.to_csv('../../data/reformatted_data/course.csv', index=False)
final_course_df = pd.merge(final_course_df, class_table, how='left', left_on='class name', right_on='class name')
final_course_df = pd.merge(final_course_df, subjects_table , how='left', left_on='subject name', right_on='subject name')
# Lấy tổng số tiết học
curriculum_framework = pd.read_csv('../../data/reformatted_data/curriculum_framework.csv')
curriculum_framework = curriculum_framework.iloc[:,[0,-1]]

final_course_df = pd.merge(final_course_df, curriculum_framework  , how='left', left_on='subject name', right_on='course name')
# del final_course_df['class name']
# del final_course_df['subject name']
final_course_df = final_course_df.reindex(columns = ['course id', 'class id', 'subject id', 'noc', 'total sessions', 'credit', 'start time', 'end time'])
final_course_df['credit'] = final_course_df['credit'].interpolate()
final_course_df['credit'] = final_course_df['credit'].astype('int8')
final_course_df = final_course_df.drop_duplicates(subset=['course id'])
final_course_df = final_course_df.reset_index(drop=True)

# batch & year
attendance = []
for i in file_names1:
    course_id = get_cource_infor(f'{folder_path1}/{i}')[2]
    batch = get_cource_infor(f'{folder_path1}/{i}')[4]
    year = get_cource_infor(f'{folder_path1}/{i}')[5]
    attendance.append([course_id, batch, year])

attendance2 = []
for i in file_names2:
    course_id = get_cource_infor_s(f'{folder_path2}/{i}')[0]
    batch = get_cource_infor_s(f'{folder_path2}/{i}')[3]
    year = get_cource_infor_s(f'{folder_path2}/{i}')[4]
    attendance2.append([course_id, batch, year])

batch_year_df1 = pd.DataFrame(attendance)
batch_year_df2 = pd.DataFrame(attendance2)
batch_year_df = pd.concat([batch_year_df1,batch_year_df2])
batch_year_df.columns = ['course id', 'batch', 'year']
batch_year_df = batch_year_df.drop_duplicates(keep='first', subset=['course id'])
batch_year_df = batch_year_df.reset_index(drop=True)
course_df = pd.merge(final_course_df, batch_year_df, how='left', right_on='course id', left_on='course id')
course_df = course_df.drop_duplicates(keep='first', subset=['course id'])
course_df['batch'] = course_df['batch'].astype(int)
course_df['year'] = course_df['year'].astype(int)
# FK
cond = course_df['start time'].isna() & course_df['end time'].isna()
# st, et = null
null_time = course_df[cond]
# st, et != null
course_df_temp = course_df.loc[~course_df.index.isin(null_time.index)][['start time', 'end time', 'batch', 'year']].drop_duplicates(keep='first', subset = ['batch', 'year'])
null_time_df = pd.merge(null_time,course_df_temp, how='left', on=['batch', 'year'])
null_time_df.set_index(null_time.index, inplace=True)
null_time_df = null_time_df.drop(columns=['start time_x','end time_x'])
null_time_df = null_time_df.rename(columns = {'start time_y':'start time',
                                              'end time_y':'end time'})
null_time_df = null_time_df.reindex(columns = ['course id','class id','subject id','noc','total sessions','credit','start time', 'end time', 'batch', 'year'])
null_time_df.loc[27, 'start time'] = '20/07/2021'
null_time_df.loc[27, 'end time'] = '22/08/2021'
# Cập nhật 'start time' và 'end time' trong course_df từ null_time_df
course_df.loc[null_time.index, ['start time', 'end time']] = null_time_df[['start time', 'end time']]
course_df['noc'] = course_df['noc'].fillna(12)
course_df['noc'] = course_df['noc'].astype(int)
course_df = course_df.rename(columns = {"course id": "course_id",
                                        "class id": "class_id",
                                        "subject id": "subject_id",
                                        "noc": "noc",
                                        "total sessions":"total_sessions",
                                        "credit": "credit",
                                        "start time": "start_time",
                                        "end time": "end_time",
                                        "batch": "batch",
                                        "year": "year"})
course_df.to_csv('../../data/reformatted_data/course.csv',mode='w', index=False)
course_df

,course_id,class_id,subject_id,noc,total_sessions,credit,start_time,end_time,batch,year
0,100107462101,0,41,6,60,3,29/07/2023,31/08/2023,3,2022
1,100107462102,1,41,6,60,3,26/07/2023,30/08/2023,3,2022
2,100107462901,1,29,12,45,3,10/10/2022,26/12/2022,1,2022
3,100107463001,0,35,12,45,3,14/02/2023,06/05/2023,2,2022
4,100107463002,1,35,12,45,3,15/02/2023,06/05/2023,2,2022
5,100107464801,7,15,6,60,3,10/10/2023,14/11/2023,1,2023
6,100107464802,6,15,6,60,3,11/10/2023,15/11/2023,1,2023
7,100107464803,5,15,6,60,3,12/10/2023,16/11/2023,1,2023
8,100107464901,1,44,6,60,3,12/10/2023,16/11/2023,1,2023
9,100107464902,0,44,6,60,3,13/10/2023,17/11/2023,1,2023


In [12]:
""" Bảng course_student """
dfs = []
dfs2 = []
for file_name in file_names1:
    df = pd.read_excel(os.path.join(folder_path1, file_name))
    course_infor = df.iloc[7, 2]
    course_id = course_infor[-24:-12]
    student_id = df.iloc[12:, 1].values
    course_student_df = pd.DataFrame({'student id': student_id, 'course id': course_id})
    dfs.append(course_student_df)
course_student_df = pd.concat(dfs, ignore_index=True)

for file_name in file_names2:
    df = pd.read_excel(os.path.join(folder_path2, file_name))
    course_id = get_cource_infor_s(f'{folder_path2}/{file_name}')[0]
    student_id = df.iloc[11:df.shape[0]-6,1]
    student_id_df = pd.DataFrame(student_id)
    student_id_df['course id'] = course_id
    dfs2.append(student_id_df)
course_student_df2 = pd.concat(dfs2, ignore_index=True)
course_student_df2 = course_student_df2.rename(columns={'Unnamed: 1': 'student id'})
course_student = pd.concat([course_student_df, course_student_df2], ignore_index=True)
course_student

,student id,course id
0,2000001008,100107627401
1,2000004387,100107627401
2,2000000998,100107627401
3,2000002380,100107627401
4,2000003127,100107627401
...,...,...
957,2200010216,100107470505
958,2200009170,100107470505
959,2200010939,100107470505
960,2200000333,100107470505


In [13]:
""" Bảng student_subject """
dfs = []
dfs2 = [] 
for file_name in file_names1:
    df = pd.read_excel(os.path.join(folder_path1, file_name))
    student_id = df.iloc[12:, 1]
    subject_name = get_cource_infor(os.path.join(folder_path1, file_name))[0]
    student_id_df = pd.DataFrame(student_id)
    student_id_df['subject name'] = subject_name.strip(' ')
    dfs.append(student_id_df)

student_subject_df = pd.concat(dfs, ignore_index=True)
student_subject_df = pd.merge(student_subject_df, subjects_table, how='left', on='subject name')
del student_subject_df['subject name']

for file_name in file_names2:
    df = pd.read_excel(os.path.join(folder_path2, file_name))
    student_id = df.iloc[11:df.shape[0]-6,1]
    subject_name = get_cource_infor_s(os.path.join(folder_path2, file_name))[2]
    student_id_df = pd.DataFrame(student_id)
    student_id_df['subject name'] = subject_name.strip(' ')
    dfs2.append(student_id_df)

student_subject_df2 = pd.concat(dfs2, ignore_index=True)
student_subject_df2 = pd.merge(student_subject_df2, subjects_table, how='left', on='subject name')
del student_subject_df2['subject name']

student_subject = pd.concat([student_subject_df,student_subject_df2], ignore_index=True)
student_subject = student_subject.rename(columns = {'Unnamed: 1':'student id'})
student_subject


,student id,subject id
0,2000001008,40
1,2000004387,40
2,2000000998,40
3,2000002380,40
4,2000003127,40
...,...,...
957,2200010216,3
958,2200009170,3
959,2200010939,3
960,2200000333,3


In [14]:
""" BẢNG INITIAL OUTCOME """
score_ls = []
for file_name in file_names2:
    df = pd.read_excel(os.path.join(folder_path2, file_name))
    course_info = df.iloc[3,4]
    course_id = (course_info.split(' - ')[0]).split(': ')[1].strip('[]')
    student_id = df.iloc[11:df.shape[0]-6,1]
    score_table = df.iloc[11:df.shape[0]-6, 7:]
    score_table = score_table.drop(['Unnamed: 9','Unnamed: 10'], axis = 1)
    score_df = pd.concat([student_id,score_table], axis = 1)
    score_df['course id'] = course_id
    score_ls.append(score_df)
score_df =  pd.concat(score_ls, ignore_index=True)
score_df.columns = ['student id', 'midterm', 'regular score', 'exam permission', 'final semester', 'note', 'course id']
score_df['score id'] = [str(i) for i in range(score_df.shape[0])]
score_df = score_df.reindex(columns=['score id','course id', 'student id', 'midterm', 'regular score', 'exam permission', 'final semester', 'note'])

def classification(x):
    if x < 4:
        return 'F'
    elif 4 <= x < 5:
        return 'D'
    elif 5 <= x < 7:
        return 'C'
    elif 7 <= x < 8.5:
        return 'B'
    elif x >= 8.5:
        return 'A'
    else:
        return 'Invalid score'
outcome_df = score_df.drop(['student id'], axis=1)
outcome_df['exam permission'] = outcome_df['exam permission'].replace(['V', np.nan], True)
outcome_df['exam permission'] = ~(outcome_df['exam permission'].isin(['V', np.nan]))
# Chuyển đổi định dạng từ dấu phẩy sang dấu chấm thập phân

# Chuyển đổi kiểu dữ liệu
outcome_df['midterm'] = outcome_df['midterm'].apply(lambda x: x.rstrip("'") if "'" in str(x) else x)
outcome_df[['final semester', 'midterm', 'regular score']] = outcome_df[['final semester', 'midterm', 'regular score']].replace({',': '.'}, regex=True)
outcome_df[['final semester', 'midterm', 'regular score']] = outcome_df[['final semester', 'midterm', 'regular score']].astype('float16')
outcome_df.to_csv('../../data/reformatted_data/outcome.csv', index=False, mode='w')


# Điểm tông kết 
final_score = (outcome_df['midterm'] * 0.3 + outcome_df['regular score']*0.2 + outcome_df['final semester']*0.5)
final_score = final_score.map(lambda x: round(x,2))
four_point_scale = final_score.map(lambda x: classification(x))
temp_df = pd.concat([final_score, four_point_scale], axis = 1)

outcome_df = pd.concat([outcome_df,temp_df], axis=1)
outcome_df = outcome_df.rename(columns = {0: 'final score', 
                                          1: 'four point scale'})

outcome_df

C:\Users\HP\AppData\Local\Temp\ipykernel_50244\520932587.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  outcome_df['exam permission'] = outcome_df['exam permission'].replace(['V', np.nan], True)


,score id,course id,midterm,regular score,exam permission,final semester,note,final score,four point scale
0,0,100107463502,10.0,10.0,True,10.0,NaN,10.00,A
1,1,100107463502,9.0,10.0,True,10.0,NaN,9.70,A
2,2,100107463502,6.0,7.0,True,8.5,NaN,7.45,B
3,3,100107463502,6.0,10.0,True,10.0,NaN,8.80,A
4,4,100107463502,10.0,7.0,True,8.5,NaN,8.65,A
...,...,...,...,...,...,...,...,...,...
266,266,100107470505,5.0,3.0,True,0.0,NaN,2.10,F
267,267,100107470505,9.0,9.0,True,8.0,NaN,8.50,A
268,268,100107470505,8.0,10.0,True,9.0,NaN,8.90,A
269,269,100107470505,10.0,9.0,True,8.0,NaN,8.80,A


In [15]:
""" Bảng initial outcome_student """
outcome_student_df = score_df.loc[:, ['score id','student id']]
outcome_student_df.to_csv('../../data/reformatted_data/outcome_student.csv', index=False, mode='w')
outcome_student_df

,score id,student id
0,0,2000001094
1,1,2000001008
2,2,2000004387
3,3,2000002177
4,4,2000002380
...,...,...
266,266,2200010216
267,267,2200009170
268,268,2200010939
269,269,2200000333


In [16]:
""" Bảng detail_attendance """ 
days_list = [] # Chứa tất cả giá trị ngày tháng năm
list_df = [] # Chứa kết quả giai đoạn 1
list_df2 = [] # Chứa kết quả của giai đoạn 2
for i in file_names1:
    df = pd.read_excel(f'../../data/gathered_data/attendance_sheet/{i}', skiprows=10)
    # ----------------------------------GIAI ĐOẠN 1 ------------------------------------------
    ''' TẠO DF CÓ SID, COURSE_CODE, DATE '''

    # Task 1: Lấy danh sách dữ liệu ngày tháng năm trong tất cả các file.
    day_df = df.iloc[:,6:]
    day_df = day_df.iloc[:, :day_df.shape[1]-4]
    days_list = []
    days_array = day_df.iloc[0].dropna().values
    for p in days_array:
        days_list.append(p[len(p)-10:])

    # Task 2: Lấy thông tin chung của khóa học từ hàm get_cource_infor được tạo ở trên
    course_infor = get_cource_infor(f'../../data/gathered_data/attendance_sheet/{i}')
    course_code = course_infor[2]
    # class_name = course_infor[1] 
    # subject_title = course_infor[0]

    # Task 3: Lấy ra date_columns - là dữ liệu điểm danh
    date_columns = df.iloc[:, 6: df.shape[1]-4][1:]
    # Format lại date_columns
    new_header = date_columns.iloc[0]
    date_columns = date_columns[1:]
    date_columns.columns = new_header
    date_columns.columns.name = None
    date_columns.reset_index(drop = True, inplace = True)

    # Task 4: Tạo ra Series chứa thông tin mã sinh viên
    sID_column = df.iloc[:,1][2:]
    sID_column.name = 'SID' # Đổi tên Series thành SID
    sID_column.reset_index(drop=True, inplace=True) # Reset index cho sID

    # Task 5: Tạo dataframe gồm SID, Course_code, Date
    big_empty_list = []
    for i in sID_column.values:
        for j in days_list:
            empty_list = [i,course_code,j]
            big_empty_list.append(empty_list)

    sID_with_date_df = pd.DataFrame(big_empty_list, columns = ['SID','Course_code', 'Date'])
    list_df.append(sID_with_date_df)
    # # ----------------------------------GIAI ĐOẠN 2 ------------------------------------------
    ''' TẠO DF CÓ CÁC CỘT: (P/K), ST, LD    
        TỪ TẤT CÁC FILE'''

    # Tạo series mã học phần có số lượng phần tử bằng với số lượng phần tử của sID_column
    course_code_sr = pd.Series([course_code] * len(sID_column))
    # Đổi tên series
    course_code_sr.name = 'Course_code'
    # Gộp series (sID_column, course_code_sr) với dataframe date_columns 
    attendance_df = pd.concat([sID_column,course_code_sr,date_columns],axis = 1)
    reshape_df = int(attendance_df.iloc[:, 2:].size / 3)
    reshaped_array = attendance_df.iloc[:, 2:].values.reshape(reshape_df,3)
    attendace_df_ = pd.DataFrame(reshaped_array, columns = ['(P/K)','ST','LD'])
    list_df2.append(attendace_df_)


    # # ----------------------------------GIAI ĐOẠN 3 ------------------------------------------
    ''' GỘP DATAFRAME GIAI ĐOẠN 1 VÀ 2'''

sID_with_date_df = pd.concat(list_df, ignore_index=True)
attendace_df_ = pd.concat(list_df2, ignore_index=True)
completed_attendace_df = pd.concat([sID_with_date_df, attendace_df_], axis = 1)
completed_attendace_df = completed_attendace_df.sort_values(by=['Course_code','Date'], ignore_index=True)
completed_attendace_df.columns = ['student id', 'course id', 'date', 'permission', 'school period', 'reason']
completed_attendace_df = completed_attendace_df.replace({'P': True, 'K':False})
completed_attendace_df['school period'] = completed_attendace_df['school period'].astype('int16')
completed_attendace_df = completed_attendace_df.sort_values(by=['course id', 'student id'])
completed_attendace_df = completed_attendace_df.rename(columns={'school period': 'absent class periods'})
def check(x):
    if x == True:
        x = 1
    elif x == False:
        x = 0
    else:
        x = 2
    return x
completed_attendace_df['permission'] = completed_attendace_df['permission'].apply(lambda x: check(x))
completed_attendace_df


C:\Users\HP\AppData\Local\Temp\ipykernel_50244\3143059613.py:71: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  completed_attendace_df = completed_attendace_df.replace({'P': True, 'K':False})


,student id,course id,date,permission,absent class periods,reason
21,1900007784,100107462101,05/08/2023,0,4,NaN
50,1900007784,100107462101,12/08/2023,0,4,NaN
79,1900007784,100107462101,19/08/2023,2,0,NaN
108,1900007784,100107462101,26/08/2023,2,0,NaN
137,1900007784,100107462101,29/07/2023,2,0,NaN
...,...,...,...,...,...,...
5076,2011551765,100107627402,11/07/2023,2,0,NaN
5102,2011551765,100107627402,13/06/2023,2,0,NaN
5128,2011551765,100107627402,18/07/2023,2,0,NaN
5154,2011551765,100107627402,20/06/2023,2,0,NaN


In [17]:
""" Attendance Stats"""
""" Tìm tỉ lệ số học sinh đi học vắng không phép"""
student_id_array = completed_attendace_df['student id'].unique()

# Số buổi vắng không phép của sinh viên
non_attended_sessions_not_p_dic = {}
non_attended_sessions_p_dic = {}
attended_sessions_dic = {}

for i in student_id_array:
    non_attendance_count_not_p = completed_attendace_df[completed_attendace_df['student id'] == f'{i}']
    non_attended_sessions_not_p = (non_attendance_count_not_p['permission'] == '0').sum()
    non_attended_sessions_not_p_dic[f'{i}'] = non_attended_sessions_not_p

# Số buổi vắng có phép của sinh viên
for i in student_id_array:
    non_attendance_count_p = completed_attendace_df[completed_attendace_df['student id'] == f'{i}']
    non_attended_sessions_p = (non_attendance_count_p['permission'] == '1').sum()
    non_attended_sessions_p_dic[f'{i}'] = non_attended_sessions_p

# Số buổi học không vắng
for i in student_id_array:
    attendance_count = completed_attendace_df[completed_attendace_df['student id'] == f'{i}']
    attended_sessions = (attendance_count['permission'].isna()).sum()
    attended_sessions_dic[f'{i}'] = attended_sessions




# Tính tổng số buổi học của sinh viên.
total_sessions_dic = {}
for i in student_id_array:
    attendance_count = completed_attendace_df[completed_attendace_df['student id'] == f'{i}']
    total_sessions = attendance_count.shape[0]
    total_sessions_dic[f'{i}'] = total_sessions


student_id_list = list(non_attended_sessions_not_p_dic.keys())
# Tỉ lệ vắng học không phép của sinh viên so với tổng số buổi học
non_attended_rate_not_p_dic = {}
# Tỉ lệ vắng học có phép của sinh viên so với tổng số buổi học
non_attended_rate_p_dic = {}
# Tỉ lệ sinh viên đi học đầy đủ.
attended_rate_dic = {}
# Tỉ lệ vắng học của sinh viên so với tổng số buổi học
absenteeism_rate_dic = {}

""" Tìm tỉ lệ số học sinh vắng học không phép"""
for student_id in student_id_list:
    
    non_attended_sessions_not_p = non_attended_sessions_not_p_dic[student_id]
    total_sessions = total_sessions_dic[student_id]

    # Tính tỉ lệ số buổi vắng học không phép của sinh viên
    if total_sessions != 0:
        non_attended_not_p_rate = round((non_attended_sessions_not_p / total_sessions)*100,1)
    else:
        non_attended_not_p_rate = 0

    non_attended_rate_not_p_dic[f'{student_id}'] = non_attended_not_p_rate


""" Tìm tỉ lệ số học sinh vắng học có phép"""
for student_id in student_id_list:
    non_attended_sessions_p = non_attended_sessions_p_dic[student_id]
    total_sessions = total_sessions_dic[student_id]

    # Tính tỉ lệ số buổi vắng học có phép của sinh viên
    if total_sessions != 0:
        non_attended_p_rate = round((non_attended_sessions_p / total_sessions) * 100,1)
    else:
        non_attended_p_rate = 0

    non_attended_rate_p_dic[f'{student_id}'] = non_attended_p_rate


""" Tìm tỉ lệ học sinh đi học không vắng"""
for student_id in student_id_list:
    attended_sessions = attended_sessions_dic[student_id]
    total_sessions = total_sessions_dic[student_id]

    # Tính tỉ lệ số buổi đi học không vắng của sinh viên
    if total_sessions != 0:
        attended_rate = round(float((attended_sessions / total_sessions) * 100), 1)
    else:
        attended_rate = 0

    attended_rate_dic[f'{student_id}'] = attended_rate


""" Tìm tỉ lệ học sinh đi học không vắng"""
for student_id in student_id_list:
    attended_sessions = attended_sessions_dic[student_id]
    total_sessions = total_sessions_dic[student_id]

    # Tính tỉ lệ số buổi đi học không vắng của sinh viên
    if total_sessions != 0:
        attended_rate = round(float((attended_sessions / total_sessions) * 100), 1)
    else:
        attended_rate = 0

    attended_rate_dic[f'{student_id}'] = attended_rate

""" Tỉ lệ vắng học của sinh viên so với tổng số buổi học"""
for student_id in student_id_list:
    absenteeism =  non_attended_sessions_not_p_dic[student_id] + non_attended_sessions_p_dic[student_id]
    total_sessions = total_sessions_dic[student_id]
    absenteeism_rate = round(float((absenteeism / total_sessions) * 100), 1)
    absenteeism_rate_dic[student_id] = absenteeism_rate



attendance_stats = pd.DataFrame({
    'excused absence rate': non_attended_rate_not_p_dic,
    'unexcused absence rate': non_attended_rate_p_dic,
    'attendance rate': attended_rate_dic,
    'absenteeism rate': absenteeism_rate_dic
})

attendance_stats['student id'] = attendance_stats.index
attendance_stats = attendance_stats.reset_index(drop=True)
attendance_stats = attendance_stats.reindex(columns=['student id','excused absence rate','unexcused absence rate','attendance rate','absenteeism rate'])
attendance_stats.to_csv('../../data/reformatted_data/overall_statistic_detail_attendance.csv', index=False, mode='w')
attendance_stats

,student id,excused absence rate,unexcused absence rate,attendance rate,absenteeism rate
0,1900007784,0.0,0.0,0.0,0.0
1,2000000050,0.0,0.0,0.0,0.0
2,2000000655,0.0,0.0,0.0,0.0
3,2000000740,0.0,0.0,0.0,0.0
4,2000001094,0.0,0.0,0.0,0.0
...,...,...,...,...,...
276,2100004451,0.0,0.0,0.0,0.0
277,2100004998,0.0,0.0,0.0,0.0
278,2100009025,0.0,0.0,0.0,0.0
279,2100010779,0.0,0.0,0.0,0.0


In [18]:
""" INITIAL ATTENDANCE STATISTICS"""
student_id_array = completed_attendace_df['student id'].unique()
attendance_list = []
for student_id in student_id_array:
    df = completed_attendace_df[completed_attendace_df['student id'] == student_id]
    course_id_array = df['course id'].unique()
    for course_id in course_id_array:
        course_id_df = df[df['course id'] == course_id]
        # Tổng số buổi học
        total_sessions = course_id_df.shape[0]
        # Tổng số tiết học.
        total_class_periods = course_df[course_df['course_id'] == course_id]['total_sessions'].values[0]
        # Tổng số tiết học vắng
        absent_class_period = course_id_df['absent class periods'].sum()
        # Tổng số buổi học vắng
        total_absense_sessions = ((course_id_df['permission'] == False) | (course_id_df['permission'] == True)).sum()
        # Tổng số buổi học vắng có phép
        excused_absense = course_id_df[course_id_df['permission'] == True]['absent class periods'].sum()
        # Tổng số buổi học vắng không phép
        unexcused_absense = course_id_df[course_id_df['permission'] == False]['absent class periods'].sum()
        # Tỉ lệ vắng học
        rate_of_absense = round(float((total_absense_sessions / total_class_periods) * 100), 1)
        # Thêm vào ds
        attendance_list.append((student_id, course_id, excused_absense, unexcused_absense, total_sessions,total_class_periods,absent_class_period, rate_of_absense))




    attendance_statistic = pd.DataFrame(attendance_list) #.transpose()
    attendance_statistic.columns = ['student id', 'course id', 'excused absense', 'unexcused absense', 'total sessions','total class periods','absent class period','absenteeism rate']
    attendance_statistic = attendance_statistic.sort_values(by=['course id'])
    attendance_statistic[['excused absense', 'unexcused absense', 'total sessions','total class periods','absent class period']] = attendance_statistic[['excused absense', 'unexcused absense', 'total sessions', 'total class periods', 'absent class period']].astype(int)
    attendance_statistic['absenteeism rate'] = attendance_statistic['absenteeism rate'].astype(float)

    attendance_df = completed_attendace_df.drop_duplicates(keep = 'first', subset=['student id', 'course id']).iloc[:, :3]
    attendance_statistic = pd.merge(attendance_statistic,attendance_df, how='left', left_on=['student id', 'course id'], right_on=['student id', 'course id'])
    attendance_statistic['attendance id'] = [str(i) for i in range(attendance_statistic.shape[0])]
    attendance_statistic = attendance_statistic.reindex(columns = ['attendance id','student id', 'course id', 'total sessions','excused absense', 'unexcused absense','absent class period','total class periods','absenteeism rate'])
    attendance_statistic.to_csv('../../data/reformatted_data/statistic_detail_attendance.csv', index=False, mode='w')
attendance_statistic

,attendance id,student id,course id,total sessions,excused absense,unexcused absense,absent class period,total class periods,absenteeism rate
0,0,1900007784,100107462101,6,0,8,8,60,3.3
1,1,2000002677,100107462101,6,0,8,8,60,3.3
2,2,2000002924,100107462101,6,1,8,9,60,5.0
3,3,2000003177,100107462101,6,1,12,13,60,6.7
4,4,2000003308,100107462101,6,0,12,12,60,5.0
...,...,...,...,...,...,...,...,...,...
686,686,2000002514,100107627402,8,0,0,0,60,0.0
687,687,2011549832,100107627402,8,0,15,15,60,8.3
688,688,2011551765,100107627402,8,0,0,0,60,0.0
689,689,2000002924,100107627402,8,0,3,3,60,1.7


In [19]:
# bảng overall statistic
temp = attendance_statistic.iloc[:,[0, 1]]
temp_lst = []
for i in attendance_stats.iloc[:, 0].values.tolist():
    temp_lst.append(temp[temp['student id'] == i])

overall_statistic = pd.concat(temp_lst, ignore_index=True)
overall_statistic.to_csv('../../data/reformatted_data/overall_statistic.csv', index=False, mode='w')

In [20]:
""" DATA OUTCOME - OUTCOME_STUDENT - ATTENDANCE STATISTICS"""
import random
# outcome_df
# outcome_student_df

# # Kiểm tra student_id trong outcome_student có bất kỳ dữ liệu nào khác với student
# student_ids_notin_final_students = outcome_student_df[~outcome_student_df['student id'].isin(final_students['student_id'])]['student id']
# # Kết quả là null -> đúng

def classification(x):
    if x < 4:
        return 'F'
    elif 4 <= x < 5:
        return 'D'
    elif 5 <= x < 7:
        return 'C'
    elif 7 <= x < 8.5:
        return 'B'
    elif x >= 8.5:
        return 'A'
    else:
        return 'Invalid score'

""" 2 trường hợp:
                  1: Có điểm danh nhưng không có điểm -> fake điểm
                  2: Có điểm nhưng không có điểm danh -> fake điểm danh"""


# 1. Trường hợp 1: Có điểm danh nhưng không có điểm. -> Tạo ra điểm cho những sinh viên chưa có điểm danh

# Tìm những cặp student id và course id còn thiếu trong outcome
current_student_course_id_pair = pd.merge(outcome_df, outcome_student_df, how='left', on='score id')[['student id', 'course id']].values
absenteeism_pairs = set(map(tuple, current_student_course_id_pair))
attendance_pairs = set(map(tuple, attendance_statistic[['student id', 'course id']].values))

# Tìm các cặp course id vaf student id có trong attendance_pairs nhưng không có trong outcome
missing_pairs = attendance_pairs - absenteeism_pairs
missing_pairs_array = np.array(list(missing_pairs))
missing_pairs_df = pd.DataFrame(missing_pairs_array, columns=['student id', 'course id'])
case1_df = attendance_statistic[(attendance_statistic['student id'].isin(missing_pairs_df['student id'])) & 
                                (attendance_statistic['course id'].isin(missing_pairs_df['course id']))]

# Tính điểm thường xuyên
absenteeism_rate_case1 = case1_df.copy()
absenteeism_rate_case1 = absenteeism_rate_case1[['student id','course id','absenteeism rate']]
absenteeism_rate_case1.loc[:, 'regular score'] = absenteeism_rate_case1['absenteeism rate'].apply(lambda x: int(100 - x) / 10)
absenteeism_rate_case1

# Thêm outcome_student_df
additional_outcome_df = absenteeism_rate_case1[['student id','course id', 'regular score']]
additional_outcome_df.index = range(271, 271 + len(additional_outcome_df))
additional_outcome_df = additional_outcome_df.reset_index()
additional_outcome_df = additional_outcome_df.rename(columns = {'index': 'score id'})

# outcome_student_df
additional_outcome_student_df = additional_outcome_df[['score id', 'student id']]

# outcome_df
additional_outcome_df['midterm'] = np.NaN
additional_outcome_df['midterm'] = additional_outcome_df['midterm'].apply(lambda x: round(random.uniform(0, 10), 1))
additional_outcome_df['final semester'] = np.NaN
additional_outcome_df['final semester'] = additional_outcome_df['final semester'].apply(lambda x: round(random.uniform(4, 10), 1))
additional_outcome_df['exam permission'] = np.where((additional_outcome_df['midterm'] <= 1) & (additional_outcome_df['regular score'] < 5.0), 0, 1)
additional_outcome_df['final score'] = (additional_outcome_df['regular score'] * 0.2 + additional_outcome_df['midterm'] * 0.3 + additional_outcome_df['final semester'] * 0.5)
additional_outcome_df['four point scale'] = additional_outcome_df['final score'].map(lambda x: classification(x))
additional_outcome_df['note'] = np.nan
additional_outcome_df = additional_outcome_df.reindex(columns = ['score id', 'course id', 'midterm', 'regular score','exam permission','final semester', 'note','final score','four point scale'])


# Ghép additional_outcome_df với outcome_df gốc
outcome = pd.concat([additional_outcome_df, outcome_df])
outcome_student = pd.concat([outcome_student_df, additional_outcome_student_df])

# 2. Trường hợp 2: Có điểm nhưng không có điểm danh.
missing_pairs2 = absenteeism_pairs - attendance_pairs
missing_pairs_array2 = np.array(list(missing_pairs2 ))
missing_pairs2_df = pd.DataFrame(missing_pairs_array2, columns = ['student id', 'course id'])

# Tạo df outcome với cột regular score
case2_df = pd.merge(outcome_df, outcome_student_df, how='left', on='score id')[['student id', 'course id', 'regular score']]
# outcome_student_df[~outcome_student_df['student id'].isin(missing_pairs2_df['student id'])] (for test)
additional_attendance_statistic = case2_df.copy()
additional_attendance_statistic['attendance id'] = range(691, 691 + len(additional_attendance_statistic))
additional_attendance_statistic['total class periods'] = np.random.choice([45, 60], size=len(additional_attendance_statistic))
additional_attendance_statistic['absenteeism rate'] = additional_attendance_statistic['regular score'].apply(lambda x: int(-(x*10)+100))


additional_attendance_statistic['eu'] = np.round((additional_attendance_statistic['absenteeism rate'] / 100) * additional_attendance_statistic['total class periods'])
additional_attendance_statistic['excused absense'] = np.random.randint(0, additional_attendance_statistic['eu'] + 1, len(additional_attendance_statistic))

additional_attendance_statistic['unexcused absense'] = additional_attendance_statistic['eu'] - additional_attendance_statistic['excused absense']
additional_attendance_statistic['unexcused absense'] = additional_attendance_statistic['unexcused absense'].astype(int)

del additional_attendance_statistic['eu']
additional_attendance_statistic['absent class period'] = additional_attendance_statistic['unexcused absense'] + additional_attendance_statistic['excused absense']
additional_attendance_statistic['total sessions'] = np.random.choice([6, 8], size=len(additional_attendance_statistic))
additional_attendance_statistic = additional_attendance_statistic.reindex(columns = ['attendance id', "student id", "course id", "total sessions", "excused absense", "unexcused absense", "absent class period", "total class periods", "absenteeism rate"])
additional_attendance_statistic
attendance_statistic_df = pd.concat([attendance_statistic, additional_attendance_statistic])
attendance_statistic_df




,attendance id,student id,course id,total sessions,excused absense,unexcused absense,absent class period,total class periods,absenteeism rate
0,0,1900007784,100107462101,6,0,8,8,60,3.3
1,1,2000002677,100107462101,6,0,8,8,60,3.3
2,2,2000002924,100107462101,6,1,8,9,60,5.0
3,3,2000003177,100107462101,6,1,12,13,60,6.7
4,4,2000003308,100107462101,6,0,12,12,60,5.0
...,...,...,...,...,...,...,...,...,...
266,957,2200010216,100107470505,8,13,18,31,45,70.0
267,958,2200009170,100107470505,8,3,3,6,60,10.0
268,959,2200010939,100107470505,8,0,0,0,60,0.0
269,960,2200000333,100107470505,6,6,0,6,60,10.0


In [21]:
print(subjects_table.dtypes)
subjects_table

subject id      object
subject name    object
dtype: object


,subject id,subject name
0,0,Tiếng Anh 2
1,1,Toán cao cấp (CNTT)
2,2,Dịch vụ Hành chính
3,3,Lập trình
4,4,Toán rời rạc
5,5,Công nghệ thông tin
6,6,Tiếng Anh 3
7,7,Giáo dục thể chất
8,8,Kỹ năng giao tiếp
9,9,Pháp luật đại cương


In [22]:
final_students = final_students.rename(columns = {'first name':'first_name',
                                                  'last name':'last_name',
                                                  'school year':'school_year'})
final_students.to_csv('../../data/reformatted_data/student.csv', index=False,mode='w')
print(final_students.dtypes)
final_students

student_id     object
first_name     object
last_name      object
gender          int64
dob            object
school_year    object
dtype: object


,student_id,first_name,last_name,gender,dob,school_year
0,2000001008,Thái Mai Hoàng,Anh,1,15/10/2002,20
1,2000004387,Trần Thanh,Bình,1,07/12/2002,20
2,2000000998,Võ Thành,Công,1,02/07/2001,20
3,2000002380,Huỳnh Hoàng,Duy,1,16/12/2002,20
4,2000003127,Phạm Trần Quốc,Duy,1,01/09/2002,20
...,...,...,...,...,...,...
271,2200002076,Võ Duy,Toàn,1,27/07/2004,22
272,2200010216,LÊ MINH,TRÍ,1,12/08/2004,22
273,2200009170,LÊ THANH,TRỌNG,1,10/06/2003,22
274,2200010939,Lê Đoàn Anh,Tuấn,1,25/09/2004,22


In [23]:
student_subject = student_subject.rename(columns = {'student id': 'student_id',
                                                    'subject id': 'subject_id'})
print(student_subject.dtypes)
student_subject.to_csv('../../data/reformatted_data/subject_student.csv', index=False, mode='w')
student_subject



student_id    object
subject_id    object
dtype: object


,student_id,subject_id
0,2000001008,40
1,2000004387,40
2,2000000998,40
3,2000002380,40
4,2000003127,40
...,...,...
957,2200010216,3
958,2200009170,3
959,2200010939,3
960,2200000333,3


In [24]:
class_table = class_table.rename(columns = {'class id': 'class_id',
                                            'class name':'class_name'})
class_table.to_csv('../../data/reformatted_data/class_section.csv', index=False,mode='w')
print(class_table.dtypes)
class_table

class_id      object
class_name    object
dtype: object


,class_id,class_name
0,0,20BITV01
1,1,20BITV02
2,2,21BITV01
3,3,21BITV02
4,4,21BITV03
5,5,22BITV02
6,6,22BITV03
7,7,22BITV04
8,8,22BITV03
9,9,22BITV04


In [25]:
print(course_df.dtypes)
course_df

course_id         object
class_id          object
subject_id        object
noc                int32
total_sessions     int64
credit              int8
start_time        object
end_time          object
batch              int32
year               int32
dtype: object


,course_id,class_id,subject_id,noc,total_sessions,credit,start_time,end_time,batch,year
0,100107462101,0,41,6,60,3,29/07/2023,31/08/2023,3,2022
1,100107462102,1,41,6,60,3,26/07/2023,30/08/2023,3,2022
2,100107462901,1,29,12,45,3,10/10/2022,26/12/2022,1,2022
3,100107463001,0,35,12,45,3,14/02/2023,06/05/2023,2,2022
4,100107463002,1,35,12,45,3,15/02/2023,06/05/2023,2,2022
5,100107464801,7,15,6,60,3,10/10/2023,14/11/2023,1,2023
6,100107464802,6,15,6,60,3,11/10/2023,15/11/2023,1,2023
7,100107464803,5,15,6,60,3,12/10/2023,16/11/2023,1,2023
8,100107464901,1,44,6,60,3,12/10/2023,16/11/2023,1,2023
9,100107464902,0,44,6,60,3,13/10/2023,17/11/2023,1,2023


In [26]:
course_student = course_student.rename({'student id':'student_id',
                                        'course id': 'course_id'})
print(course_student.dtypes)
course_student.to_csv('../../data/reformatted_data/course_student.csv', index=False, mode='w')
course_student


student id    object
course id     object
dtype: object


,student id,course id
0,2000001008,100107627401
1,2000004387,100107627401
2,2000000998,100107627401
3,2000002380,100107627401
4,2000003127,100107627401
...,...,...
957,2200010216,100107470505
958,2200009170,100107470505
959,2200010939,100107470505
960,2200000333,100107470505


In [27]:
del attendance_statistic_df['attendance id']
attendance_statistic_df.to_csv('../../data/reformatted_data/statistic_detail_attendance.csv', index=False, mode='w')
print(attendance_statistic_df.dtypes)
attendance_statistic_df = attendance_statistic_df.rename(columns = {'student id':'student_id',
                                                                    'course id': 'course_id'})
attendance_statistic_df

student id              object
course id               object
total sessions           int32
excused absense          int32
unexcused absense        int32
absent class period      int32
total class periods      int32
absenteeism rate       float64
dtype: object


,student_id,course_id,total sessions,excused absense,unexcused absense,absent class period,total class periods,absenteeism rate
0,1900007784,100107462101,6,0,8,8,60,3.3
1,2000002677,100107462101,6,0,8,8,60,3.3
2,2000002924,100107462101,6,1,8,9,60,5.0
3,2000003177,100107462101,6,1,12,13,60,6.7
4,2000003308,100107462101,6,0,12,12,60,5.0
...,...,...,...,...,...,...,...,...
266,2200010216,100107470505,8,13,18,31,45,70.0
267,2200009170,100107470505,8,3,3,6,60,10.0
268,2200010939,100107470505,8,0,0,0,60,0.0
269,2200000333,100107470505,6,6,0,6,60,10.0


In [28]:
outcome_student.to_csv('../../data/reformatted_data/outcome_student.csv', index=False, mode='w')
print(outcome_student.dtypes)
outcome_student

score id      object
student id    object
dtype: object


,score id,student id
0,0,2000001094
1,1,2000001008
2,2,2000004387
3,3,2000002177
4,4,2000002380
...,...,...
686,957,2000002514
687,958,2011549832
688,959,2011551765
689,960,2000002924


In [29]:
outcome = outcome.rename(columns = {'score id': 'outcome_id',
                                    'course id': 'course_id',
                                    'midterm': 'midterm_grade',
                                    'regular score': 'regular grade',
                                    'exam permission': 'exam_permission',
                                    'final semester': 'final_grade',
                                    'final score': 'overall_grade',
                                    'four point scale': 'four_point scale'})
outcome.to_csv('../../data/reformatted_data/outcome.csv', index=False, mode='w')
print(outcome.dtypes)
outcome

outcome_id           object
course_id            object
midterm_grade       float64
regular grade       float64
exam_permission       int32
final_grade         float64
note                 object
overall_grade       float64
four_point scale     object
dtype: object


,outcome_id,course_id,midterm_grade,regular grade,exam_permission,final_grade,note,overall_grade,four_point scale
0,271,100107462101,9.2,9.6,1,8.0,NaN,8.68,A
1,272,100107462101,1.6,9.6,1,6.5,NaN,5.65,C
2,273,100107462101,3.2,9.5,1,10.0,NaN,7.86,B
3,274,100107462101,1.4,9.3,1,8.9,NaN,6.73,C
4,275,100107462101,6.4,9.5,1,4.0,NaN,5.82,C
...,...,...,...,...,...,...,...,...,...
266,266,100107470505,5.0,3.0,1,0.0,NaN,2.10,F
267,267,100107470505,9.0,9.0,1,8.0,NaN,8.50,A
268,268,100107470505,8.0,10.0,1,9.0,NaN,8.90,A
269,269,100107470505,10.0,9.0,1,8.0,NaN,8.80,A


In [30]:
completed_attendace_df['attendance_id'] = range(0,completed_attendace_df.shape[0])
completed_attendace_df = completed_attendace_df.rename(columns = {'student id': 'student_id',
                                                                  'course id': 'course_id',
                                                                  'absent class periods':'absent_class_periods'})
completed_attendace_df = completed_attendace_df.reindex(columns = ['attendance_id', 'student_id','course_id','date', 'permission','absent_class_periods','reason'])
completed_attendace_df.to_csv('../../data/reformatted_data/detail_attendance.csv', index=False, mode='w')
print(completed_attendace_df.dtypes)
completed_attendace_df


attendance_id            int64
student_id              object
course_id               object
date                    object
permission               int64
absent_class_periods     int16
reason                  object
dtype: object


,attendance_id,student_id,course_id,date,permission,absent_class_periods,reason
21,0,1900007784,100107462101,05/08/2023,0,4,NaN
50,1,1900007784,100107462101,12/08/2023,0,4,NaN
79,2,1900007784,100107462101,19/08/2023,2,0,NaN
108,3,1900007784,100107462101,26/08/2023,2,0,NaN
137,4,1900007784,100107462101,29/07/2023,2,0,NaN
...,...,...,...,...,...,...,...
5076,5181,2011551765,100107627402,11/07/2023,2,0,NaN
5102,5182,2011551765,100107627402,13/06/2023,2,0,NaN
5128,5183,2011551765,100107627402,18/07/2023,2,0,NaN
5154,5184,2011551765,100107627402,20/06/2023,2,0,NaN


In [31]:
df = pd.merge(outcome, outcome_student, how='left', left_on='outcome_id', right_on='score id')
df.drop(columns=['outcome_id', 'note'], inplace=True)
df = df.rename(columns = {'student id':'student_id'})
non_pre_processed_df = pd.merge(df, attendance_statistic_df, how = 'left', on=['student_id', 'course_id'])

non_pre_processed_df = non_pre_processed_df.reindex(columns = ['student_id','course_id','regular grade', 'absent class period','absenteeism rate','midterm_grade', 'final_grade','overall_grade', 'four_point scale'])
non_pre_processed_df = non_pre_processed_df.rename(columns = {'regular grade':'regular_grade',
                                                          'absent class period': 'absent_class_period',
                                                          'absenteeism rate': 'absenteeism_rate',
                                                          'four_point scale': 'four_point_scale'
                                                          })

non_pre_processed_df = pd.merge(non_pre_processed_df, course_df, how = 'left', on = ['course_id'])
non_pre_processed_df = pd.merge(non_pre_processed_df, final_students, how = 'left', on = ['student_id'])



In [32]:
non_pre_processed_df = non_pre_processed_df.reindex(columns = ['student_id','course_id', 'subject_id', 'gender','dob','regular_grade', 'absent_class_period','absenteeism_rate','midterm_grade', 'final_grade','overall_grade', 'four_point_scale', 'credit','batch', 'year'])

def calculate_age(x):
    age = datetime.datetime.now().year - int(x[-4:])
    return age


non_pre_processed_df['dob'] = non_pre_processed_df['dob'].apply(lambda x: calculate_age(x))
non_pre_processed_df = non_pre_processed_df.rename(columns = {'dob': 'age'})
non_pre_processed_df

,student_id,course_id,subject_id,gender,age,regular_grade,absent_class_period,absenteeism_rate,midterm_grade,final_grade,overall_grade,four_point_scale,credit,batch,year
0,1900007784,100107462101,41,1,23,9.6,8,3.3,9.2,8.0,8.68,A,3,3,2022
1,2000002677,100107462101,41,0,26,9.6,8,3.3,1.6,6.5,5.65,C,3,3,2022
2,2000002924,100107462101,41,1,22,9.5,9,5.0,3.2,10.0,7.86,B,3,3,2022
3,2000003177,100107462101,41,1,23,9.3,13,6.7,1.4,8.9,6.73,C,3,3,2022
4,2000003308,100107462101,41,1,22,9.5,12,5.0,6.4,4.0,5.82,C,3,3,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,2200010216,100107470505,3,1,20,3.0,31,70.0,5.0,0.0,2.10,F,3,2,2022
958,2200009170,100107470505,3,1,21,9.0,6,10.0,9.0,8.0,8.50,A,3,2,2022
959,2200010939,100107470505,3,1,20,10.0,0,0.0,8.0,9.0,8.90,A,3,2,2022
960,2200000333,100107470505,3,0,21,9.0,6,10.0,10.0,8.0,8.80,A,3,2,2022


In [33]:
non_pre_processed_df['semester'] = non_pre_processed_df['batch'].astype(str) + non_pre_processed_df['year'].astype(str)
non_pre_processed_df.drop(columns = ['batch', 'year'])
non_pre_processed_df

,student_id,course_id,subject_id,gender,age,regular_grade,absent_class_period,absenteeism_rate,midterm_grade,final_grade,overall_grade,four_point_scale,credit,batch,year,semester
0,1900007784,100107462101,41,1,23,9.6,8,3.3,9.2,8.0,8.68,A,3,3,2022,32022
1,2000002677,100107462101,41,0,26,9.6,8,3.3,1.6,6.5,5.65,C,3,3,2022,32022
2,2000002924,100107462101,41,1,22,9.5,9,5.0,3.2,10.0,7.86,B,3,3,2022,32022
3,2000003177,100107462101,41,1,23,9.3,13,6.7,1.4,8.9,6.73,C,3,3,2022,32022
4,2000003308,100107462101,41,1,22,9.5,12,5.0,6.4,4.0,5.82,C,3,3,2022,32022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,2200010216,100107470505,3,1,20,3.0,31,70.0,5.0,0.0,2.10,F,3,2,2022,22022
958,2200009170,100107470505,3,1,21,9.0,6,10.0,9.0,8.0,8.50,A,3,2,2022,22022
959,2200010939,100107470505,3,1,20,10.0,0,0.0,8.0,9.0,8.90,A,3,2,2022,22022
960,2200000333,100107470505,3,0,21,9.0,6,10.0,10.0,8.0,8.80,A,3,2,2022,22022


In [34]:
# Xóa học kỳ dư.
non_pre_processed_df['semester'].value_counts()
non_pre_processed_df.drop(index = non_pre_processed_df[non_pre_processed_df['semester']=='32021'].index, inplace=True)


In [35]:
# Tính điểm trung bình học kỳ:
def gpa(group):
    # Tính điểm trung bình môn học có trọng số trong một học kỳ
    gpa = (group['overall_grade'] * group['credit']).sum() / group['credit'].sum()
    return gpa

student_semester_gpa = non_pre_processed_df.groupby(['student_id','semester'], as_index=False).apply(lambda x: round(gpa(x[['overall_grade', 'credit']]), 1), include_groups=False)
student_semester_gpa = student_semester_gpa.rename(columns = {None:'gpa'})
student_semester_gpa 


,student_id,semester,gpa
0,1900006098,22022,6.1
1,1900006098,32022,6.6
2,1900007549,12022,5.9
3,1900007763,12022,7.5
4,1900007784,12023,8.4
...,...,...,...
669,2200011643,12023,6.8
670,2200011643,22022,8.0
671,2200011704,22022,7.3
672,2200011866,12023,5.9


In [36]:
df = student_semester_gpa.pivot_table(index='student_id', columns='semester', values='gpa')
df = df.reset_index()
df.rename(columns = {'12022':'gpa1',
                     '22022': 'gpa2',
                     '32022':'gpa3',
                     '12023':'gpa4'}, inplace = True)
df = df.reindex(columns=['student_id','gpa1','gpa2','gpa3','gpa4'])

In [37]:
# Điền giá trị thiếu gpa1
missing_indices = df['gpa1'].isnull()
random_values = np.random.uniform(low=3, high=10, size=missing_indices.sum()).round(1)
df.loc[missing_indices, 'gpa1'] = random_values

# Điền giá trị thiếu gpa2
missing_indices = df['gpa2'].isnull()
random_values = np.random.uniform(low=3, high=10, size=missing_indices.sum()).round(1)
df.loc[missing_indices, 'gpa2'] = random_values

# Điền giá trị thiếu gpa3
missing_indices = df['gpa3'].isnull()
random_values = np.random.uniform(low=3, high=10, size=missing_indices.sum()).round(1)
df.loc[missing_indices, 'gpa3'] = random_values

# Điền giá trị thiếu gpa2
missing_indices = df['gpa4'].isnull()
random_values = np.random.uniform(low=3, high=10, size=missing_indices.sum()).round(1)
df.loc[missing_indices, 'gpa4'] = random_values
df


semester,student_id,gpa1,gpa2,gpa3,gpa4
0,1900006098,9.5,6.1,6.6,5.5
1,1900007549,5.9,7.7,7.7,7.5
2,1900007763,7.5,4.6,4.5,9.5
3,1900007784,7.6,7.1,7.3,8.4
4,1900008106,6.7,7.8,4.7,4.2
...,...,...,...,...,...
320,2200011621,8.1,2.7,9.5,6.5
321,2200011643,7.8,8.0,9.6,6.8
322,2200011704,10.0,7.3,9.8,9.2
323,2200011866,7.6,7.2,9.4,5.9


In [38]:
absenteeism_rate = non_pre_processed_df.pivot_table(index='student_id', columns='semester', values='absenteeism_rate')
absenteeism_rate = absenteeism_rate	.reset_index()
absenteeism_rate.rename(columns = 
                    {
                     '12022':'ar1',
                     '12023': 'ar4',
                     '22022':'ar2',
                     '32022':'ar3'}, inplace = True)
absenteeism_rate = absenteeism_rate.reindex(columns=['student_id','ar1','ar2','ar3','ar4'])

# Điền giá trị thiếu gpa1
missing_indices = absenteeism_rate['ar1'].isnull()
random_values = np.random.uniform(low=0, high=20, size=missing_indices.sum()).round(1)
absenteeism_rate.loc[missing_indices, 'ar1'] = random_values

missing_indices = absenteeism_rate['ar2'].isnull()
random_values = np.random.uniform(low=0, high=20, size=missing_indices.sum()).round(1)
absenteeism_rate.loc[missing_indices, 'ar2'] = random_values

missing_indices = absenteeism_rate['ar3'].isnull()
random_values = np.random.uniform(low=0, high=20, size=missing_indices.sum()).round(1)
absenteeism_rate.loc[missing_indices, 'ar3'] = random_values

missing_indices = absenteeism_rate['ar4'].isnull()
random_values = np.random.uniform(low=0, high=20, size=missing_indices.sum()).round(1)
absenteeism_rate.loc[missing_indices, 'ar4'] = random_values

In [39]:
merge_df = pd.merge(df, absenteeism_rate, on='student_id', how='left')
merge_df

semester,student_id,gpa1,gpa2,gpa3,gpa4,ar1,ar2,ar3,ar4
0,1900006098,9.5,6.1,6.6,5.5,14.6,2.2,35.0,19.5
1,1900007549,5.9,7.7,7.7,7.5,0.0,18.0,0.0,19.5
2,1900007763,7.5,4.6,4.5,9.5,0.0,12.3,0.0,7.2
3,1900007784,7.6,7.1,7.3,8.4,5.7,2.2,16.1,8.9
4,1900008106,6.7,7.8,4.7,4.2,0.0,0.0,10.3,8.2
...,...,...,...,...,...,...,...,...,...
320,2200011621,8.1,2.7,9.5,6.5,3.2,40.0,2.2,12.1
321,2200011643,7.8,8.0,9.6,6.8,14.5,0.0,11.0,0.0
322,2200011704,10.0,7.3,9.8,9.2,17.0,10.0,3.4,8.5
323,2200011866,7.6,7.2,9.4,5.9,10.5,1.6,10.0,0.0


In [40]:
merge_df.loc[merge_df[merge_df['student_id']=='2200000327'].index, ['ar1','ar2','ar3','ar4']] = [25.0,32.5,45.0,73.0]
merge_df.loc[merge_df[merge_df['student_id']=='2200000327'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='2000004897'].index, ['ar1','ar2','ar3','ar4']] = [57.0,60.0,27.0,45.0]
merge_df.loc[merge_df[merge_df['student_id']=='2000004897'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='2000000050'].index, ['ar1','ar2','ar3','ar4']] = [67.2,56.0,75.0,80.0]
merge_df.loc[merge_df[merge_df['student_id']=='2000000050'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='2200006775'].index, ['ar1','ar2','ar3','ar4']] = [88.0,59.2,57.0,53.0]
merge_df.loc[merge_df[merge_df['student_id']=='2200006775'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='1900006098'].index, ['ar1','ar2','ar3','ar4']] = [34.5,12.0,0.0,53.0]
merge_df.loc[merge_df[merge_df['student_id']=='1900006098'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='2200000327'].index, ['ar1','ar2','ar3','ar4']] = [25.0,32.5,45.0,73.0]
merge_df.loc[merge_df[merge_df['student_id']=='2200000327'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='2000000655'].index, ['ar1','ar2','ar3','ar4']] = [43.0,32.5,45.0,22.5]
merge_df.loc[merge_df[merge_df['student_id']=='2000000655'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()

merge_df.loc[merge_df[merge_df['student_id']=='2100006285'].index, ['ar1','ar2','ar3','ar4']] = [5.0,3.5,2,22.5]
merge_df.loc[merge_df[merge_df['student_id']=='2100006285'].index, ['gpa1','gpa2','gpa3','gpa4']] = np.random.uniform(low=0, high=4, size=4).round(1).tolist()


def dropout(row):
    if(np.all(row[['gpa1', 'gpa2', 'gpa3', 'gpa4']] < 4.0) or np.all(row[['ar1', 'ar2', 'ar3', 'ar4']] >= 50.0) or np.all(row[['ar1', 'ar2', 'ar3', 'ar4']] == 100.0)):
        return 1
    return 0


merge_df['drop_out'] = merge_df.apply(dropout, axis=1)

df = pd.merge(merge_df, final_students, how = 'left', on='student_id')
df['dob'] = df['dob'].apply(lambda x: calculate_age(x))
df = df.reindex(columns = ['student_id', 'gender', 'dob', 'gpa1','gpa2','gpa3','gpa4', 'ar1','ar2','ar3','ar4','drop_out'])
df = df.rename(columns = {'dob': 'age'})
df.to_csv('../../data/reformatted_data/non_pre_processed_data.csv', index=False, mode='w')



In [41]:
df[df['drop_out'] == 1]

,student_id,gender,age,gpa1,gpa2,gpa3,gpa4,ar1,ar2,ar3,ar4,drop_out
0,1900006098,1,23,2.3,0.2,2.5,3.2,34.5,12.0,0.0,53.0,1
15,2000000050,1,24,1.3,3.7,2.8,3.2,67.2,56.0,75.0,80.0,1
17,2000000655,1,22,1.6,2.0,1.0,1.6,43.0,32.5,45.0,22.5,1
66,2000004897,1,22,1.0,3.6,3.1,2.5,57.0,60.0,27.0,45.0,1
122,2100006285,1,21,0.8,1.4,1.0,3.1,5.0,3.5,2.0,22.5,1
166,2200000327,1,20,1.4,1.3,3.2,3.3,25.0,32.5,45.0,73.0,1
259,2200006775,1,20,1.5,1.6,0.6,3.7,88.0,59.2,57.0,53.0,1
